# **Alter Ego**

### **Description**
Even so, You alone should drown in blue and vanish.\
Before being consumed by those sorrow-filled eyes, divide the world by zero, humming with a hoarse voice.\
The finale’s sound is ringing out.\
Author: kanon

In [2]:
from Crypto.Util.number import *

from random import randint
import os

from sage.all import *

from montgomery_isogenies.kummer_line import KummerLine
from montgomery_isogenies.kummer_isogeny import KummerLineIsogeny


### **1. Các hằng số:**

- `FLAG`: Lấy cờ và encode thành bytes, nếu không tồn tại, sử dụng giá trị mặc định
- `proof.arithmetic(False)`: Tắt chế độ kiểm tra chứng minh của các phép toán số học, tăng tốc độ tính toán.
- `MI`, `KU`, `MIKU`: Là các biến, chưa biết được sử dụng làm gì
- `ells`: Danh sách 74 số nguyên tố lẻ đầu tiên. 
- `p`: $p = 4 * \prod_i \text{ells}_i - 1$. Hiểu đơn giản, `p` là một số nguyên tố lớn (64 byte = 521 bit)
- `Fp = GF(p)`: Tạo trường hữu hạn cơ sở $\mathbb{F}_p$
- `F = GF(p**2, modulus=x**2 + 1, names='i')`: Tạo trường mở rộng bậc 2: $\mathbb{F}_{p^2} = \mathbb{F}_p[i]/(i^2+1)$, tức thêm phần tử $i$ sao cho $i^2 = -1$
- `i = F.gen(0)`: Gán generator `i` này để dùng trong các phép tính.
- `E0 = EllipticCurve(F, [1, 0])`: Tạo đường cong elliptic $\mathbb{E}_0$ trên trường $\mathbb{F}_{p^2}$ với phương trình:
$$
    y^{2} = x^3 + x
$$
- `E0.set_order((p + 1)**2)` chỉ định (gán) rằng đường cong elliptic $E_0$ có tổng số điểm (order của nhóm $E_{0}(\mathbb{F}_{p^2}) = (p+1)^2$ 

In [3]:
# FLAG = os.getenv('flag', "SEKAI{With_zero_oxygen_remaining,_I_want_to_be_able_to_say_'that_was_an_amazing_game'_with_my_final_breath._https://youtu.be/qo6e2tiVEeo}").encode()
FLAG = os.getenv('flag', "SEKAI{here_is_test_flag_hehe}").encode()

proof.arithmetic(False)

MI = 3
KU = 9
MIKU = 39

ells = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 587]

p = 4 * prod(ells) - 1

Fp = GF(p)
F = GF(p**2, modulus=x**2 + 1, names='i')
i = F.gen(0)

E0 = EllipticCurve(F, [1, 0])           # Ax + B
E0.set_order((p + 1)**2)                

### **2. Phương thức `group_action`**

#### Mục đích
Phương thức `group_action` được sử dụng để **thực hiện chuỗi các phép đẳng cấu isogeny trên một đường cong elliptic (hoặc Kummer line)**, dựa trên **vector khóa bí mật `priv`**. Kết quả của hàm là **một đường cong mới** (đã được biến đổi qua isogenies) và **một điểm mới tương ứng trên Kummer line**.

Hàm này là một phần trong các giao thức **isogeny-based cryptography** (như SIDH, CSIDH, SQISign…), nơi mà hành động của một nhóm abelian trên tập hợp các đường cong được sử dụng để sinh khóa hoặc mã hóa.

---
#### 🧠 Giải thích từng thành phần

| Thành phần | Giải thích |
|-------------|-------------|
| `_C` | Đường cong ban đầu (Kummer line hoặc elliptic curve) |
| `priv` | Vector khóa bí mật, mỗi phần tử biểu thị số lần áp dụng isogeny theo hướng dương hoặc âm |
| `G` | Một điểm trên đường cong (được dùng để theo dõi quá trình biến đổi qua các isogeny) |
| `es = priv[:]` | Sao chép danh sách `priv` để có thể thay đổi trong quá trình tính toán mà không ảnh hưởng đến dữ liệu gốc |
| `Fp.random_element()` | Lấy một phần tử ngẫu nhiên trong trường hữu hạn $ \mathbb{F}_p $, dùng để chọn toạ độ $x$ cho điểm $P$ |
| `P = _C(x)` | Tạo một điểm $P$ trên đường cong Kummer hoặc elliptic tương ứng với hoành độ $x$ |
| `A = _C.curve().a2()` | Lấy hệ số $a_2$ của phương trình đường cong $y^2 = x^3 + a_2x^2 + a_4x + a_6$ |
| `s = 1 if Fp(x ^ 3 + A * x ^ 2 + x).is_square() else -1` | Xác định dấu (sign) của phép biến đổi: nếu biểu thức bên trong là **bình phương trong $ \mathbb{F}_p $** thì `s = +1`, ngược lại `s = -1`. Dấu này cho biết hướng của isogeny cần thực hiện. |
| `S = [i for i, e in enumerate(es) if sign(e) == s and e != 0]` | Chọn danh sách các chỉ số `i` trong `es` mà có cùng dấu với `s` (nghĩa là sẽ được xử lý trong vòng lặp này) |
| `k = prod([ells[i] for i in S])` | Tính tích các số nguyên tố `ells[i]` tương ứng với các chỉ số `S`. Đây là bậc của phép nhân điểm ban đầu $Q = \frac{p+1}{k}P$. |
| `Q = int((p + 1) // k) * P` | Tạo điểm $Q$ từ $P$ bằng cách nhân với $\frac{p+1}{k}$. Điểm này được dùng làm đầu vào cho chuỗi isogeny. |
| `for i in S:` | Duyệt qua từng chỉ số `i` trong danh sách `S` để áp dụng lần lượt các isogeny bậc `ells[i]`. |
| `R = (k // ells[i]) * Q` | Tạo điểm $R$ là nhân của $Q$, sao cho bậc của $R$ chính là `ells[i]`. $R$ được dùng làm kernel cho isogeny bậc `ells[i]`. |
| `if R.is_zero(): continue` | Nếu $R$ là điểm 0, thì bỏ qua (vì không thể tạo isogeny từ điểm 0). |
| `phi = KummerLineIsogeny(_C, R, ells[i])` | Xây dựng **isogeny bậc `ells[i]`** trên Kummer line, sử dụng điểm $R$ làm kernel. |
| `_C = phi.codomain()` | Cập nhật lại đường cong hiện tại `_C` thành **đường cong ảnh của isogeny** `phi`. |
| `Q, G = phi(Q), phi(G)` | Áp dụng ánh xạ isogeny lên các điểm $Q$ và $G$, chuyển chúng sang đường cong mới. |
| `es[i] -= s` | Giảm phần tử tương ứng trong `es` đi 1 theo hướng `s`, biểu thị rằng một lần isogeny bậc `ells[i]` đã được thực hiện. |
| `k //= ells[i]` | Giảm giá trị `k` tương ứng, vì đã thực hiện một thành phần trong tích. |
| `while any(es):` | Vòng lặp tiếp tục cho đến khi tất cả giá trị trong `es` đều bằng 0, nghĩa là tất cả isogeny cần thực hiện đã xong. |
| `return _C, G` | Trả về kết quả cuối cùng gồm: đường cong sau khi áp dụng toàn bộ isogeny và điểm tương ứng $G$ đã được biến đổi. |


#### 📘 Ví dụ minh họa
Để hiểu rõ cách hoạt động của hàm `group_action`, ta sẽ xây dựng một ví dụ mô phỏng từng bước với các tham số nhỏ và dễ quan sát.

```python
# Khởi tạo trường và tham số
p = 101
Fp = GF(p)
ells = [3, 5]              # danh sách các bậc isogeny (số nguyên tố nhỏ)
priv = [1, -1]             # vector khóa bí mật, gồm 2 phần tử

# Tạo đường cong elliptic y² = x³ + x trên Fp
E = EllipticCurve(Fp, [0, 0, 0, 1, 0]) # 0xy + 0x^2 + 0y + x + 0
C = KummerLine(E)          # ánh xạ sang dạng Kummer line (ẩn đi tọa độ y)

# Tạo một điểm ngẫu nhiên G trên Kummer line
G = C(E.random_point())

# Gọi hàm group_action
final_curve, final_G = group_action(C, priv, G)

print("a2 của đường cong kết quả:", final_curve.curve().a2())
print("Điểm kết quả trên Kummer line:", final_G)
```
🧩 Giải thích từng bước
1. *Khởi tạo đầu vào*
- `p = 101`: Là số nguyên tố xác định trường hữu hạn $\mathbb{F}_p$
- `ells = [3, 5]`: Danh sách các bậc isogeny sẽ được dùng (ở đây là 3 và 5).
- `priv = [1, -1]`: Khóa bí mật mô tả số lần áp dụng mỗi isogeny, và hướng (+ hoặc -).
    + 1 → thực hiện isogeny bậc 3 theo hướng dương (+).
    + -1 → thực hiện isogeny bậc 5 theo hướng âm (-).

2. *Tạo đường cong và điểm*
- Đường cong $E: y^2 = x^3 + x$
- Dạng Kummer line của nó là $C$, chỉ giữ lại thông tin về tọa độ $x$.
- `G` là điểm ngẫu nhiên trên Kummer line, dùng để “theo dõi” qua các isogeny.

3. *Bắt đầu vòng lặp chính*
- Sao chép `priv` sang `es` để làm việc mà không ảnh hưởng đến dữ liệu gốc.
- Kiểm tra `while any(es)`: → tiếp tục cho đến khi tất cả giá trị trong `es` đều bằng 0.

4. *Trong mỗi vòng lặp:*
- Sinh ngẫu nhiên `x = Fp.random_element()`.
- Tạo điểm `P = C(x)` trên đường cong.
- Lấy hệ số `A = C.curve().a2()`.
- Tính giá trị `s = 1` hoặc `-1`:
    * `s = 1 if Fp(x ^ 3 + A * x ^ 2 + x).is_square() else -1`  
        Nếu biểu thức là bình phương trong $\mathbb{F}_p$, ta chọn hướng dương, ngược lại âm.
- Lấy danh sách chỉ số cần xử lý:
    * `S = [i for i, e in enumerate(es) if sign(e) == s and e != 0]`
        Danh sách này gồm những phần tử của khóa bí mật có cùng dấu với s.
- Tính tích các bậc tương ứng: `k = prod([ells[i] for i in S])`
- Tạo điểm $$Q = \frac{p+1}{k}P$$
    `Q = int((p + 1) // k) * P`
- Với mỗi phần tử `i` trong `S`:
    + Tạo điểm kernel $R = \frac{k}{\text{ells}[i]}Q$
    + Nếu $R$ không phải là điểm $0$, xây dựng isogeny bậc `ells[i]`: `phi = KummerLineIsogeny(C, R, ells[i])`
    + Cập nhật:
    ```python
        C = phi.codomain()  # đường cong ảnh
        Q, G = phi(Q), phi(G)
        es[i] -= s          # giảm giá trị e[i]
        k /= ells[i]
    ```
5. *Kết thúc*
Khi mọi phần tử trong es = 0, ta đã thực hiện hết chuỗi isogeny. Hàm trả về: `return _C, G`\
Tức là đường cong kết quả và điểm tương ứng sau tất cả các biến đổi.
```bash
a2 của đường cong kết quả: 45
Điểm kết quả trên Kummer line: (17 : 1)
```

In [4]:
def group_action(_C, priv, G):
    es = priv[:]
    while any(es):
        x = Fp.random_element()
        P = _C(x)
        A = _C.curve().a2()
        s = 1 if Fp(x ^ 3 + A * x ^ 2 + x).is_square() else -1

        S = [i for i, e in enumerate(es) if sign(e) == s and e != 0]
        k = prod([ells[i] for i in S])
        Q = int((p + 1) // k) * P
        for i in S:
            R = (k // ells[i]) * Q
            if R.is_zero():
                continue

            phi = KummerLineIsogeny(_C, R, ells[i])
            _C = phi.codomain()
            Q, G = phi(Q), phi(G)
            es[i] -= s
            k //= ells[i]

    return _C, G

## **3. Phương thức `BEAM(base_alice_priv)`**

Phương thức được sử dụng trong **isogeny-based cryptography**, thường xuất hiện trong các bài **CTF crypto challenge**.\
Hàm này mô phỏng một quá trình tương tác giữa hai bên: *Alice* (chương trình) và *Bob*.

Hàm `BEAM()` là một vòng lặp tương tác mô phỏng *isogeny-based group action*, trong đó mỗi vòng người dùng có thể "chỉnh sửa" trạng thái khóa của Alice thông qua phản ứng tương tác.

------------------------------------------------------------------------

### ***3.1. 🧩 Tổng quan***

Hàm `BEAM(base_alice_priv)` thực hiện lặp nhiều vòng.\
Ở mỗi vòng:

1.  Tạo một **đường cong elliptic** với tham số `pub`.
2.  Biến đổi nó sang **Kummer line** để chỉ làm việc với hoành độ
    (x-only).
3.  Áp dụng **group action** với khóa riêng của Alice (`alice_priv`).
4.  In kết quả và chờ người dùng nhập phản hồi (một vector nhỏ).
5.  Cập nhật khóa riêng của Alice và lặp lại.

Sau `MIKU` vòng (ví dụ 39), chương trình kết thúc.

------------------------------------------------------------------------

### ***3.2. 🔍 Phân tích từng phần***

``` python
def BEAM(base_alice_priv):
    alice_priv = base_alice_priv
    pub = 0
```

-   `base_alice_priv`: vector khóa riêng ban đầu.
-   `pub`: giá trị khởi tạo cho tham số `a2` của đường cong elliptic.

------------------------------------------------------------------------

#### 🧮 1. Tạo đường cong elliptic

``` python
E = EllipticCurve(F, [0, pub, 0, 1, 0])
```

-   Khởi tạo đường cong elliptic trên trường `F`.
-   Dạng tổng quát: $ y^2 = x^3 + a_2 x^2 + a_4 x + a_6 $
-   Ở đây: `a2 = pub`, `a4 = 1`, `a6 = 0`.

`pub` điều khiển hình dạng của đường cong ở mỗi vòng.

------------------------------------------------------------------------

#### 💫 2. Chuyển sang Kummer Line

``` python
omae_E = KummerLine(E)
```

-   `KummerLine(E)` là biểu diễn x-only của đường cong elliptic.
-   Bỏ đối xứng `P ↦ -P`, giúp tăng tốc độ tính toán.

------------------------------------------------------------------------

#### 🎲 3. Sinh điểm ngẫu nhiên và ánh xạ

``` python
G = E.random_point()
_G = omae_E(G)
```

-   `G`: điểm ngẫu nhiên trên đường cong.
-   `_G`: ảnh của `G` trên Kummer line (chỉ dùng x).

------------------------------------------------------------------------

#### ⚙️ 4. Thực hiện group action

``` python
_final_E1, _final_G = group_action(omae_E, alice_priv, _G)
```

-   `group_action`: phép biến đổi đường cong qua isogeny, dùng secret
    key `alice_priv`.
-   Kết quả:
    -   `_final_E1`: đường cong mới.
    -   `_final_G`: điểm tương ứng trên đường cong mới.

------------------------------------------------------------------------

#### 🧾 5. In kết quả

``` python
print(f"final_a2 = {_final_E1.curve().a2()}")
print(f"{_final_G=}")
```

-   In ra hệ số `a2` của đường cong mới.
-   In ra điểm `_final_G`.

Người dùng dựa vào thông tin này để tính phản ứng (`omae_priv`).

------------------------------------------------------------------------

#### 💡 6. Nhập phản hồi từ người dùng

``` python
omae_priv = list(map(int, input("your priv >").split(", ")))
```

-   Bạn nhập danh sách số nguyên, ví dụ:

        your priv > 1, 0, -1, 0, 0, 0, 0

------------------------------------------------------------------------

#### 🔒 7. Kiểm tra hợp lệ

``` python
assert all([abs(pi) < 2 for pi in omae_priv])
assert len(omae_priv) == len(ells)
```

-   Mỗi phần tử chỉ được là `-1`, `0`, hoặc `1`.
-   Độ dài bằng số lượng phần tử trong `ells`.

------------------------------------------------------------------------

#### 🔁 8. Cập nhật khóa riêng

``` python
alice_priv = [ai + yi for ai, yi in zip(alice_priv, omae_priv)]
print("updated")
```

-   Cập nhật khóa riêng của Alice bằng cách cộng từng phần tử.

------------------------------------------------------------------------

#### 🔁 9. Cập nhật tham số đường cong

``` python
pub = _final_E1.curve().a2()
```

-   `a2` của đường cong mới sẽ là `pub` cho vòng tiếp theo.

------------------------------------------------------------------------

#### ✅ 10. Kết thúc

``` python
print("FIN!")
```

------------------------------------------------------------------------

## 🧠 Minh họa luồng logic

    (base_alice_priv)
            ↓
       vòng 1: E(pub=0)
            ↓ group_action(alice_priv)
       → output (E1, G1)
            ↓ người chơi nhập omae_priv
       → alice_priv ← alice_priv + omae_priv
            ↓
       vòng 2: E(pub = E1.a2)
            ↓ ...
       ...
       vòng 39 → FIN!




In [ ]:
def BEAM(base_alice_priv):
    alice_priv = base_alice_priv

    pub = 0

    for _ in range(MIKU):

        E = EllipticCurve(F, [0, pub, 0, 1, 0])
        omae_E = KummerLine(E)
        G = E.random_point()
        _G = omae_E(G)

        _final_E1, _final_G = group_action(omae_E, alice_priv, _G)
        _final_G = _final_G
        print(f"final_a2 = {_final_E1.curve().a2()}")
        print(f"{_final_G=}")

        omae_priv = list(map(int, input("your priv >").split(", ")))

        assert all([abs(pi) < 2 for pi in omae_priv])
        assert len(omae_priv) == len(ells)

        alice_priv = [ai + yi for ai, yi in zip(alice_priv, omae_priv)]
        print("updated")

        pub = _final_E1.curve().a2()
    print("FIN!")


Mục tiêu của đoạn code: Kiểm tra xem bạn có thể tìm được một vector `alter_ego` (gần như “khóa bí mật thay thế”) sao cho hai group action khác nhau (`alice_priv` và `alter_ego`) tạo ra cùng một đường cong elliptic.

In [ ]:
if __name__ == "__main__":

    print("And now, it's time for the moment you've been waiting for!")

    alice_priv = [randint(MI + KU, MI * KU) for _ in ells]
    BEAM(alice_priv)

    alter_ego = list(map(int, input('ready?! here is the "alter ego" >').split(", ")))

    assert alice_priv != alter_ego
    assert len(alice_priv) == len(alter_ego)
    assert all([-MI * KU <= ai < 0 for ai in alter_ego])

    _E0 = KummerLine(E0)
    G = E0.random_point()
    _G = _E0(G)

    _alter_ego_E1, _ = group_action(_E0, alter_ego, _G)
    _alice_E1, __ = group_action(_E0, alice_priv, _G)

    if _alter_ego_E1.curve().a2() == _alice_E1.curve().a2():
        print("There you are... I've been waiting and waiting for you to come to me.")
        print(FLAG)
    else:
        print("YOU CANT FIND MY ALTER EGO....")
        exit()